#### Packages

In [243]:
import pandas as pd
import numpy as np

#### Input

In [244]:
df_SIN = pd.read_csv("data_CNCF_SIN.csv")
df_TWN = pd.read_csv("data_CNCF_TWN.csv")



### 資料處理

In [245]:
y_values = [110, 120, 130, 140, 150, 160, 170, 180, 190]
immediate_x_values = [1, 2, 4, 8]
delayed_x_values = [5, 6, 8, 12]

#### SIN

In [246]:
# Initialize an empty DataFrame to store the transformed data
transformed_data = {}

# Iterate over each row to transform the data as specified
for index, row in df_TWN.iterrows():
    participant_id = row['Participant']
    a = row['a']
    b = row['b']
    c = row['c']  # Value of 'a + b' which is also equal to 'c' as given
    d = row['d']
    
    # Determine the timeframe type and construct column names based on conditions
    if b == 0 and a in [1, 2, 4, 8]:
        # Immediate timeframe
        for y in y_values:
            col_name = f"immediate_week{c}_{y}"
            # Check the value of 'd' against the tens digit of 'y'
            if d < ((y-100) // 10):
                value = "now"
            else:
                value = "future"
            
            # Add or update the entry in the transformed_data dictionary
            transformed_data.setdefault(participant_id, {})[col_name] = value
    
    elif b == 4 and a in [1, 2, 4, 8]:
        # Delayed timeframe
        for y in y_values:
            col_name = f"delayed_week{c}_{y}"
            # Check the value of 'd' against the tens digit of 'y'
            if d < ((y-100) // 10):
                value = "now"
            else:
                value = "future"
            
            # Add or update the entry in the transformed_data dictionary
            transformed_data.setdefault(participant_id, {})[col_name] = value

In [247]:
# Convert the transformed data dictionary into a DataFrame
transformed_SIN = pd.DataFrame.from_dict(transformed_data, orient='index').reset_index()
transformed_SIN.rename(columns={'index': 'ID'}, inplace=True)

##### 欄位順序

In [248]:
# Generate ordered columns
ordered_columns = ['ID']  # Start with 'ID' column
# Add immediate and delayed columns in the specified order
for timeframe in ["immediate", "delayed"]:
    x_values = immediate_x_values if timeframe == "immediate" else delayed_x_values
    for x in x_values:
        for y in y_values:
            ordered_columns.append(f"{timeframe}_week{x}_{y}")

# Reindex the DataFrame with the ordered columns (filling missing columns with NaN if necessary)
transformed_SIN = transformed_SIN.reindex(columns=ordered_columns)

##### 問卷欄位

In [249]:
transformed_SIN = transformed_SIN.merge(
    df_SIN[['participant', 'treatment', 'gender', 'race', 'econ', 'course', 'immi', 'identity', 'r1', 'l1', 'pro1', 'pro2', 'often1', 'often2']],
    left_on='ID', right_on='participant', how='left'
)

# Set the values for each column based on the merged columns
transformed_SIN['treatment'] = transformed_SIN['treatment'].apply(lambda x: "will" if x == "CF" else "no will")
transformed_SIN['race'] = transformed_SIN['race'].str.rstrip(';')
transformed_SIN['major'] = transformed_SIN['econ'].apply(lambda x: "Econ" if x == "Yes" else "Business")
transformed_SIN['year'] = transformed_SIN['course']
transformed_SIN['immigration_status'] = transformed_SIN['immi']
transformed_SIN['identity'] = transformed_SIN.apply(lambda row: row['r1'] if row['identity'] == "Others" else row['identity'], axis=1)
transformed_SIN['language'] = transformed_SIN['l1']
transformed_SIN['pro_in_English'] = transformed_SIN['pro1']
transformed_SIN['pro_in_Mandarin'] = transformed_SIN['pro2']
transformed_SIN['often_use_English'] = transformed_SIN['often1']
transformed_SIN['often_use_Mandarin'] = transformed_SIN['often2']

# Drop the redundant columns used for merging
transformed_SIN = transformed_SIN.drop(columns=['participant', 'econ', 'course', 'immi', 'r1', 'l1', 'pro1', 'pro2', 'often1', 'often2'])
transformed_SIN = transformed_SIN.drop_duplicates()


##### Output

In [250]:
print(transformed_SIN.info())

<class 'pandas.core.frame.DataFrame'>
Index: 155 entries, 0 to 1232
Data columns (total 85 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   155 non-null    int64  
 1   immediate_week1_110  155 non-null    object 
 2   immediate_week1_120  155 non-null    object 
 3   immediate_week1_130  155 non-null    object 
 4   immediate_week1_140  155 non-null    object 
 5   immediate_week1_150  155 non-null    object 
 6   immediate_week1_160  155 non-null    object 
 7   immediate_week1_170  155 non-null    object 
 8   immediate_week1_180  155 non-null    object 
 9   immediate_week1_190  155 non-null    object 
 10  immediate_week2_110  155 non-null    object 
 11  immediate_week2_120  155 non-null    object 
 12  immediate_week2_130  155 non-null    object 
 13  immediate_week2_140  155 non-null    object 
 14  immediate_week2_150  155 non-null    object 
 15  immediate_week2_160  155 non-null    object 

In [251]:
transformed_SIN.to_excel("LSH_SIN.xlsx", index=False)

#### TWN

In [252]:
# Initialize an empty DataFrame to store the transformed data
transformed_data = {}

# Iterate over each row to transform the data as specified
for index, row in df_TWN.iterrows():
    participant_id = row['Participant']
    a = row['a']
    b = row['b']
    c = row['c']  # Value of 'a + b' which is also equal to 'c' as given
    d = row['d']
    
    # Determine the timeframe type and construct column names based on conditions
    if b == 0 and a in [1, 2, 4, 8]:
        # Immediate timeframe
        for y in y_values:
            col_name = f"immediate_week{c}_{y}"
            # Check the value of 'd' against the tens digit of 'y'
            if d < ((y-100) // 10):
                value = "now"
            else:
                value = "future"
            
            # Add or update the entry in the transformed_data dictionary
            transformed_data.setdefault(participant_id, {})[col_name] = value
    
    elif b == 4 and a in [1, 2, 4, 8]:
        # Delayed timeframe
        for y in y_values:
            col_name = f"delayed_week{c}_{y}"
            # Check the value of 'd' against the tens digit of 'y'
            if d < ((y-100) // 10):
                value = "now"
            else:
                value = "future"
            
            # Add or update the entry in the transformed_data dictionary
            transformed_data.setdefault(participant_id, {})[col_name] = value

In [253]:
# Convert the transformed data dictionary into a DataFrame
transformed_TWN = pd.DataFrame.from_dict(transformed_data, orient='index').reset_index()
transformed_TWN.rename(columns={'index': 'ID'}, inplace=True)

In [254]:
# Generate ordered columns
ordered_columns = ['ID']  # Start with 'ID' column
# Add immediate and delayed columns in the specified order
for timeframe in ["immediate", "delayed"]:
    x_values = immediate_x_values if timeframe == "immediate" else delayed_x_values
    for x in x_values:
        for y in y_values:
            ordered_columns.append(f"{timeframe}_week{x}_{y}")

# Reindex the DataFrame with the ordered columns (filling missing columns with NaN if necessary)
transformed_TWN = transformed_TWN.reindex(columns=ordered_columns)

In [255]:
transformed_TWN = transformed_TWN.merge(
    df_TWN[['Participant', 'Treatment', 'Gender', 'Taiwan', 'Econ', 'Course', 'laugnauge', 'otherLanguage', 'WhichLanguage', 'EngCountry', 'WhichCountry']],
    left_on='ID', right_on='Participant', how='left'
)

# Set the values for each column based on the merged columns
transformed_TWN['treatment'] = transformed_TWN['Treatment'].apply(lambda x: "will" if x == "CF" else "no will")
transformed_TWN['gender'] = transformed_TWN['Gender']
transformed_TWN['race'] = transformed_TWN['Taiwan']
transformed_TWN['major'] = transformed_TWN['Econ'].apply(lambda x: "Econ" if x == "是" else "Others")
transformed_TWN['year'] = transformed_TWN['Course']
transformed_TWN['language'] = transformed_TWN.apply(lambda row: "中文" if row['otherLanguage'] == "否" else row['WhichLanguage'], axis=1)
transformed_TWN['english_speaking_country'] = transformed_TWN.apply(lambda row: row['EngCountry'] if row['EngCountry'] == "否" else row['WhichCountry'], axis=1)

# Drop the redundant columns used for merging
transformed_TWN = transformed_TWN.drop(columns=['Participant', 'Treatment', 'Gender', 'Taiwan', 'Econ', 'Course', 'laugnauge', 'otherLanguage', 'WhichLanguage', 'EngCountry', 'WhichCountry'])
transformed_TWN = transformed_TWN.drop_duplicates()


In [256]:
print(transformed_TWN.info())

<class 'pandas.core.frame.DataFrame'>
Index: 155 entries, 0 to 1232
Data columns (total 80 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ID                        155 non-null    int64 
 1   immediate_week1_110       155 non-null    object
 2   immediate_week1_120       155 non-null    object
 3   immediate_week1_130       155 non-null    object
 4   immediate_week1_140       155 non-null    object
 5   immediate_week1_150       155 non-null    object
 6   immediate_week1_160       155 non-null    object
 7   immediate_week1_170       155 non-null    object
 8   immediate_week1_180       155 non-null    object
 9   immediate_week1_190       155 non-null    object
 10  immediate_week2_110       155 non-null    object
 11  immediate_week2_120       155 non-null    object
 12  immediate_week2_130       155 non-null    object
 13  immediate_week2_140       155 non-null    object
 14  immediate_week2_150       155 

In [257]:
transformed_TWN.to_excel("LSH_TWN.xlsx", index=False)

In [258]:
df_TWN.head()

,Unnamed: 0,Treatment,Location,SessionID,FutureTense,Mandarin,ID,Participant,Period,a,...,Econmajor,laugnauge,otherLanguage,WhichLanguage,EngCountry,WhichCountry,Purpose,AE,AF,chineseismainlanguage0english1ma
0,0,CN,0,160517_1320,0,1,1,1,8,8,...,1,中文,否,NaN,否,NaN,NaN,NaN,NaN,1.0
1,1,CN,0,160517_1320,0,1,1,1,7,4,...,1,中文,否,NaN,否,NaN,NaN,NaN,NaN,1.0
2,2,CN,0,160517_1320,0,1,1,1,6,1,...,1,中文,否,NaN,否,NaN,NaN,NaN,NaN,1.0
3,3,CN,0,160517_1320,0,1,1,1,5,4,...,1,中文,否,NaN,否,NaN,NaN,NaN,NaN,1.0
4,4,CN,0,160517_1320,0,1,1,1,4,8,...,1,中文,否,NaN,否,NaN,NaN,NaN,NaN,1.0
